# Welcome to the Fermionic package

## Why Julia?
I first wrote this code in Methematica, and was intrigued by performance and usage comparison with Python. There were many advantages of writing the program in Python, like defining classes and method, which was something very akward to emulate in Mathematica. The drawback was, nontheless, that Python was 10 times slower than Mathematica. This is no surprise, as Mathematica compiles directly in C language, whereas Python needs to go through numpy in order to access C functionalities (Cython was not an option, as I wanted the source code to be crystal clear to the user). 
Then I heard about Julia. I started learning the language and found it to be extremelly clear to the user, even more than Python. I run the same code and it was 10 times faster than Mathematica, and 100 times faster than Python. Besides, Julia uses something called "multiple dispatch", which replaces the traditional object oriented programming (single dispatch) and is really handy for using the same method with different types of objects. So I decided to try out this language and was more than satisfied with the results.



## Installing Julia and Juno

Lets first cover the setup needed for executing the package.

1. First of all, you must download and install the latest Julia version. You can find it in [this link](https://julialang.org/downloads/). 
2. Once installed, it is necessary to install some interpreter. I strongly recommend using [Juno](https://junolab.org/), which is a package for Atom. You must first download and install [Atom](https://atom.io/). Then from settings, you just type **uber-juno** and wait as many packages are installed. Alternativly, you can download [JuliaPro](https://juliacomputing.com/products/juliapro) that replaces Juno (you will still need Atom).
3. If everythin is working by now, you are ready. If you are working inside Atom, I recommend installing packages directly from the Juno console. You must first type ']' to access "pkg" mode. Then, just type ```add Fermionic``` and the package will be downloaded.
4. If installation was correct, you can start using all the features of the packege by importing it by typing ```using Fermionic ```

In [1]:
using Fermionic

# Operators

We then create elements of the type Op, with dimension n. 
This structure contains the operators as methods. In this package, names changes as so.

$c_i \rightarrow cm(i)\\
c_i^{\dagger} \rightarrow cmd(i)\\
c_ic_j \rightarrow cmcm(i,j)\\
c_i^{\dagger}c_j^\dagger \rightarrow cdcd(i,j)\\
c_ic_j^\dagger \rightarrow cmcd(i,j)\\
c_i^\dagger c_j \rightarrow cdcm(i)$

$\{c_i,c_j\} = \{c_i^\dagger, c_j^\dagger\} = 0\\
\{c_i,c_j^\dagger\} = \delta_{i,j}$

One of the great advantages of defining these operators as types, is that it is possible to simultaneously define operators for different dimensions. This can be useful for a number of application, for example for comparing results in different dimensions without re-running the core program each time you switch dimension.

In Object oriented programming, one would define a class Operator. Here, we are defining a type (like Int or Float) and method associated to that kind of type. The big advantage of doing this over defining classes, is that Julia uses **multiple dispatch**. What that means, is that a method can be defined for different input types, and Julia will run the correct method depending on all the input's types. Whereas in object oriented programming, programs select the method from the type of the first input only. This will be very useful for defining states, where we can work both with normal arrays and sparse arrays.

You can do, for instance

op4 = Op(4)

op6 = Op(6)

and op4 will be an element of the type ::Op, and will have all its methods defined for fermionic operators in dimension 4.

In [2]:
op4 = Op(4);

All the operators can be represented by matrices. Lets first look at the basis

In [3]:
basis(op4)

16×4 SparseArrays.SparseMatrixCSC{Float64,Int64} with 32 stored entries:
  [9 , 1]  =  1.0
  [10, 1]  =  1.0
  [11, 1]  =  1.0
  [12, 1]  =  1.0
  [13, 1]  =  1.0
  [14, 1]  =  1.0
  [15, 1]  =  1.0
  [16, 1]  =  1.0
  [5 , 2]  =  1.0
  [6 , 2]  =  1.0
  [7 , 2]  =  1.0
  [8 , 2]  =  1.0
  ⋮
  [8 , 3]  =  1.0
  [11, 3]  =  1.0
  [12, 3]  =  1.0
  [15, 3]  =  1.0
  [16, 3]  =  1.0
  [2 , 4]  =  1.0
  [4 , 4]  =  1.0
  [6 , 4]  =  1.0
  [8 , 4]  =  1.0
  [10, 4]  =  1.0
  [12, 4]  =  1.0
  [14, 4]  =  1.0
  [16, 4]  =  1.0

A sparse matrix is a way of representing a matrix in which one inidicates (row, col) = value. This is really handy for defining matrices with many zeros, as no unnecessary memory is used for storing empty elements. The drawback of the sparse representation is that it's hard to understand what is really doing. We can use the function Matrix() in order to convert a sparse matrix into a normal matrix. I recommend doing this only in order to see what is happening, not for doing operations. This is because both memory and time increase enourmusly when operations are done with full matrices instead of sparse.

In [4]:
Matrix(basis(op4))

16×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  0.0  1.0  1.0
 1.0  1.0  0.0  0.0
 1.0  1.0  0.0  1.0
 1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0

We can see that this basis is composed of 16 4-dimensional arrays, each representing a posible fermionic slater determinant in 4 dimensions ($2^n$ n-dimensional arrays for dimension n). Each mode can be occupied by 1 or by 0 fermions (due to its antisimmetric nature). It can be shown, taken the fermionic conmutation relations into account, that these states constitute an ortonormal base. Fermionic operators will be described in this basis. 

For example, let's see the matrix describing $c_i$

In [5]:
Matrix(cm(op4,1))

16×16 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

This is the matrix representing the destruction operator corresponding to the mode 1 in 4 dimensions. We can see that when it is applied to the ninth element (1,0,0,0) it takes it to the first element (0,0,0,0). When applied to (1,0,1,1) it goes to (0,0,1,1), and so on. A similar representation can be found for creation operators $c_i^\dagger$

In [6]:
Matrix(cdm(op4,4))

16×16 Array{Float64,2}:
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 1.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  -1.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0  -1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0

The minus sign in some operators is a result of the anticonmutation relations in fermions.

We can also define some one body operators, such as $c_i^\dagger c_j$:

In [7]:
Matrix(cdcm(op4,2,3))

16×16 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Of course these compund operators can be also obtained by explicitly computing the product:

In [8]:
cdcm(op4,2,3) == cdm(op4,2)*cm(op4,3)

true

When both arguments are equal, this is the number operator (counting ocupation of the i mode)

In [9]:
cdcm(op4,2,2)

16×16 SparseArrays.SparseMatrixCSC{Float64,Int64} with 8 stored entries:
  [5 ,  5]  =  1.0
  [6 ,  6]  =  1.0
  [7 ,  7]  =  1.0
  [8 ,  8]  =  1.0
  [13, 13]  =  1.0
  [14, 14]  =  1.0
  [15, 15]  =  1.0
  [16, 16]  =  1.0

One body operators defined are

$c_ic_j \rightarrow cmcm(i,j)\\
c_i^{\dagger}c_j^\dagger \rightarrow cdcd(i,j)\\
c_ic_j^\dagger \rightarrow cmcd(i,j)\\
c_i^\dagger c_j \rightarrow cdcm(i)$


# States

States are vectors that indicate the coefficient for each element in basis. In this program they can be used both as sparse arrays or as normal arrys. Let's refresh the shape of basis:

In [10]:
Matrix(basis(op4))

16×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  0.0  1.0  1.0
 1.0  1.0  0.0  0.0
 1.0  1.0  0.0  1.0
 1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0

We can clearly see that the first element corresponds to a completly empty state. This is the vacuum, and can be of coursed accesed directly with a method. 
For swithcing from sparse to regular vectors,  we now use Array()

In [11]:
vacuum(op4)

16-element SparseArrays.SparseVector{Float64,Int64} with 1 stored entry:
  [1 ]  =  1.0

In [12]:
Array(vacuum(op4))

16-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Basis has $2^n$ elements. Then a state of the shape 
state = [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0] corresponds to occupying the only the third mode

In [13]:
cdm3 = cdm(op4,3);
vac = vacuum(op4)
cdm3*vac

16-element SparseArrays.SparseVector{Float64,Int64} with 1 stored entry:
  [3 ]  =  1.0

We can naturally work with more fermions. For example, let's occupy the first three modes:

In [14]:
cdm1 = cdm(op4,1);
cdm2 = cdm(op4,2);
cdm3 = cdm(op4,3);

cdm1*cdm2*cdm3*vac

16-element SparseArrays.SparseVector{Float64,Int64} with 1 stored entry:
  [15]  =  1.0

We can make more complicated things. For example, lets can generate a random state of 2 fermions 

In [15]:
nume = 2
l = length(vac)

random_state = [0.0 for i in 1:l]

for i in 1:l
    if sum(basis(op4)[i,:]) == nume
        random_state[i] = 2*rand(Float64,1)[1]-1
    end
end

print(random_state)

[0.0, 0.0, 0.0, 0.6418155717119598, 0.0, -0.10611287368424493, -0.0014281916045129073, 0.0, 0.0, 0.40795367059771204, -0.9588264032226723, 0.0, 0.7004634307863471, 0.0, 0.0, 0.0]

We must normalize the state to norm 1.

In [16]:
random_state = random_state/sqrt(random_state'*random_state);
random_state'*random_state

1.0

Now we can initialize state as an element of the type State in order to acces more information about it

In [17]:
ran_state = State(random_state, op4);

The method belonging to this type are now avaiable:
    
- rhosp(): the one body density matrix, which is the matrix with the one body operator contractions, i.e. $\rho^{\rm sp}(i,j) = \langle \psi | c_j^\dagger c_i |\psi\rangle$.
- eigensp(): the eigenvalues of the rhosp matrix.
- ssp(): the one body entropy which is defined as 
$S(\rho^{\rm sp}) = -\sum_i (\lambda_i \log(\lambda_i) + (1-\lambda_i) \log(1-\lambda_i))$
accounting both for particle ($\lambda_i$) and holes ($1-\lambda_i$).

In [18]:
rhosp(ran_state)

4×4 Array{Float64,2}:
  0.788364   -0.0209639   0.131441   0.344926
 -0.0209639   0.251004   -0.369936   0.143364
  0.131441   -0.369936    0.665768  -0.19554
  0.344926    0.143364   -0.19554    0.294864

In [19]:
eigensp(ran_state)

4-element Array{Float64,1}:
 0.03112465894389868
 0.031124658943898736
 0.9688753410561011
 0.9688753410561011

In [20]:
ssp(ran_state)

0.5545204762433158